In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import soundfile as sf
import torch
from time import time
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self").eval()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import os
files = os.listdir('/home/nikkokks/Desktop/Project_Alexa/Audio/SPEECH_RECOGNITION/notebooks/data_pipeline/16000_1/DTLN_DPTNET/')

In [10]:
minimum = 9999999
with torch.no_grad():
    liste_denoised = []
    for file in files:
        print(file)
        ds = sf.read('/home/nikkokks/Desktop/Project_Alexa/Audio/SPEECH_RECOGNITION/notebooks/data_pipeline/16000_1/DTLN_DPTNET/'+file)
        input_values = processor(ds[0], return_tensors="pt", padding="longest",num_process=6,sampling_rate=16000).input_values  # Batch size 1
        output =  model(input_values)
        output = torch.argmax(output.logits, dim=-1)
        output = output.detach().to(device='cpu', dtype=torch.long)
        transcription = processor.batch_decode(output)
        print(transcription)
        liste_denoised.append(transcription)

p226_002.wav
['ASK HER TO BRING THESE THINGS WITH HER FROM THE STORE']
p226_009.wav
['THERE IS ACCORDING TO A LEGEND A BOILING POT OF GOLD AT ONE END']
p226_007.wav
['THE RAINBOW IS A DIVISION OF WHITE LIGHT INTO MANY BEAUTIFUL COLORS']
p226_003.wav
['SIX SPOONS OF FRESH SNOW PEAS FIVE THICK SLABS OF BLUE CHEESE AND MAYBE A SNACK FOR HER BROTHER BOB']
p226_006.wav
['WHEN THE SUNLIGHT STRIKES RAINDROPS IN THE AIR THEY ACT AS A PRISM AND FORM A RAINBOW']
p226_001.wav
['PLEASE CALL STELLA']
p226_005.wav
['SHE CAN SCOOP THESE THINGS INTO THREE RED BAGS AND WE WILL GO MEET HER WEDNESDAY AT THE TRAIN STATION']
p226_004.wav
['WE ALSO NEED A SMALL PLASTIC SNAKE AND A BIG TOY FROG FOR THE KIDS']
p226_010.wav
['PEOPLE LOOK BUT NO ONE EVER FINDS IT']
p226_008.wav
['THESE TAKE THE SHAPE OF A LONG ROUND ARCH WITH ITS PATH HIGH ABOVE AND ITS TWO ENDS APPARENTLY BEYOND THE HORIZON']


In [12]:
minimum = 9999999
with torch.no_grad():
    liste_denoised = []
    for file in files:
        print(file)
        ds = sf.read('/home/nikkokks/Desktop/Project_Alexa/Audio/SPEECH_RECOGNITION/notebooks/data_pipeline/16000_1/DTLN_DPTNET/'+file)
        input_values = processor(ds[0], return_tensors="pt", padding="longest",num_process=6,sampling_rate=16000).input_values  # Batch size 1
        output =  model(input_values)
        output = torch.argmax(output.logits, dim=-1)
        output = output.detach().to(device='cpu', dtype=torch.long)
        transcription = processor.batch_decode(output)
        print(transcription)
        liste_denoised.append(transcription)

p226_002.wav
['ASK HER TO BRING THESE THINGS WITH HER IN THE STORYT']
p226_009.wav
['THERE IS ACCORDING TO A LEGEND A BOILING POT OF GOLD AT ONE END']
p226_007.wav
['THE RAINBOW IS A DIVISION OF WHITE LIGHT INTO MANY BEAUTIFUL COLORS']
p226_003.wav
['B SIX SPOONS OF TH BREAKFAST NOW DONENTIL IT RRIVED FIVE SIX LABERLY AT MOCHEN NELL AND MAYBE A SNAPBOY HER BROTHER FOREO']
p226_006.wav
['WHEN THE SUNLIGHT STRIKES RAINDROPS IN THE AIR THEY ACT AS A PRISM AND FORM A RAINBERRY']
p226_001.wav
['PLEASE CALL STELLA']
p226_005.wav
['SHE CAN SCOOP THESE THINGS INTO THREE RED BAGS AND WE WILL GO MEET HER WEDNESDAY AT THE TRAIN STATION']
p226_004.wav
['O MI ONED NO A BUTTER A TRAL AND O BIGE TOY FROCAA  O']
p226_010.wav
['PEOPLE LOOK BUT NO ONE EVER FINDS IT']
p226_008.wav
['I SEEMED TO TAKE THE SHAPE OF A LONG ROUND ARCH WITH ITS PAT HIGH ABOVE AND ITS TWO ENDS APPARENTLY BEYOND THE HORIZON']


In [ ]:
minimum = 9999999
with torch.no_grad():
    liste_clean = []
    for file in files:
        print(file)
        ds = sf.read('clean_trainset_28spk_wav_FLAC/'+file)
        input_values = processor(ds[0], return_tensors="pt", padding="longest",num_process=6,sampling_rate=16000).input_values  # Batch size 1
        output =  model(input_values)
        output = torch.argmax(output.logits, dim=-1)
        output = output.detach().to(device='cpu', dtype=torch.long)
        transcription = processor.batch_decode(output)
        print(transcription)
        liste_clean.append(transcription)

In [ ]:
import numpy

def get_word_error_rate(r, h):

    """
    Given two list of strings how many word error rate(insert, delete or substitution).
    """
    d = numpy.zeros((len(r) + 1) * (len(h) + 1), dtype=numpy.uint16)
    d = d.reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        for j in range(len(h) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
    result = float(d[len(r)][len(h)]) / len(r) * 100

    return result


In [ ]:
liste_results = []
for i in range(len(liste_clean)):
    liste_results.append(get_word_error_rate(liste_denoised[i][0].split(),liste_clean[i][0].split()))

In [ ]:
liste_results

In [ ]:
np.mean(liste_results) # special

In [ ]:
np.mean(liste_results) # wav2vec2 base

In [ ]:
np.mean(liste_results) # wav2vec2 large